In [1]:
import os
from functools import partial
from operator import itemgetter
from dotenv import load_dotenv
import hashlib

_ = load_dotenv('.env')

In [2]:
import cassio

In [3]:
cassio.init(
    token=os.environ['ASTRA_DB_APPLICATION_TOKEN'],
    database_id=os.environ['ASTRA_DB_ID'],
    keyspace=os.environ.get('ASTRA_DB_KEYSPACE'),
)

In [4]:
# we need those as we'll run 'bare CQL' here
session = cassio.config.resolve_session()
keyspace = cassio.config.resolve_keyspace()

# Toward Hybrid search

Problem: we have several text snippets which will be vector-searched.

But it will be apparent that "vector search is not enough". What we'll explore here are solutions for hybrid search that scale well at Astra DB scale.

In [5]:
snippets = [
    "I would like to buy gift cards. Where can I get discounts?",
    "The support operator is using foul language.",
    "I cannot open the support chat.",
    "I see no messages in the support chat.",
    "Are special offers available?",
    "An operator chats with several people at the same time?",
    "A message disappeared from the chat?",
    "The support chat on the website is lagging.",
    "I cannot speak with the support operator!",
    "The operator is giving useless messages.",
    "I want to inquire about a specific product line.",
    "Is there any special offer today?",
    "I have tried multiple times to make a payment but it does not get processed.",
    "I am having trouble opening my shopping cart!",
    "Speaking to a technicial is impossible, WTF?",
]

We are intentionally leaving out any field such as "metadata", to try and focus exclusively on the text. In other words, adding metadata to the ingested corpus would require an effort (sudh as manual/AI-assisted labeling) which would scale with the number of rows, which we are trying to avoid by shifting all the load on the query side.

In [6]:
vector_dimension = 1536  # openAI ...

In [7]:
CREATE_CQL = f"CREATE TABLE {keyspace}.snippets (snippet_id TEXT PRIMARY KEY, snippet TEXT, embedding VECTOR<FLOAT,{vector_dimension}>);"
session.execute(CREATE_CQL)

CREATE_V_IDX = f"CREATE CUSTOM INDEX snippets_embedding_idx ON {keyspace}.snippets (embedding) USING 'StorageAttachedIndex';"
session.execute(CREATE_V_IDX)

## Get embeddings and insert rows

In [8]:
import openai

embedding_model_name = "text-embedding-ada-002"

def get_embeddings(texts):
    result = openai.Embedding.create(
        input=texts,
        engine=embedding_model_name,
    )
    return [res.embedding for res in result.data]

In [9]:
def snippet_id(sn): return hashlib.md5(sn.encode()).hexdigest()

print(snippet_id("Test snippet."))

50f0866734db8ec79171ddc6b13988d9


In [10]:
# TODO if ever needed, add batching to this.

embeddings = get_embeddings(snippets)

INSERT_ROW = session.prepare(f"INSERT INTO {keyspace}.snippets (snippet_id, snippet, embedding) VALUES (?, ?, ?);")

for snippet, embedding in zip(snippets, embeddings):
    session.execute(INSERT_ROW, (
        snippet_id(snippet),
        snippet,
        embedding,
    ))

## Simple retrieval

In [11]:
SIMPLE_ANN = session.prepare(f"SELECT snippet, similarity_cosine(embedding, ?) as similarity FROM {keyspace}.snippets ORDER BY embedding ANN OF ? LIMIT ?")

def simple_ann(query, top_k=3):
    q_vector = get_embeddings([query])[0]
    return [
        (row.snippet, row.similarity)
        for row in session.execute(SIMPLE_ANN, (
            q_vector,
            q_vector,
            top_k,
        ))
    ]

In [12]:
def show(results):
    for ri, (sn, si) in enumerate(results):
        print(f"    [{ri+1}] {si:.5f} \"{sn}\"")

In [13]:
QUERY = "I cannot even use the frigging website!"
print(f"QUERY: '{QUERY}'")
show(simple_ann(QUERY, 5))

QUERY: 'I cannot even use the frigging website!'
    [1] 0.92651 "I am having trouble opening my shopping cart!"
    [2] 0.92547 "I cannot speak with the support operator!"
    [3] 0.91488 "I cannot open the support chat."
    [4] 0.91049 "The support chat on the website is lagging."
    [5] 0.90430 "Speaking to a technicial is impossible, WTF?"


In [14]:
QUERY2 = "It seems that the website is broken"
print(f"QUERY2: '{QUERY2}'")
show(simple_ann(QUERY2, 5))

QUERY2: 'It seems that the website is broken'
    [1] 0.91717 "The support chat on the website is lagging."
    [2] 0.90485 "I am having trouble opening my shopping cart!"
    [3] 0.90108 "I cannot open the support chat."
    [4] 0.90091 "I see no messages in the support chat."
    [5] 0.89362 "Speaking to a technicial is impossible, WTF?"


#### Lesson: beware of setting a threshold on just-ANN and calling it a day!

## Supplemental indexing for Hybrid

General idea: adding a stemming tokenized index on the `snippet` column, and then run hybrid queries of some sort.

Let's try to use the "stemming + untouched query" case from the previous part of this journey:

In [15]:
# Don't mind the "{{" and "}}", it's just to escape the F-string syntax here
CREATE_S_IDX = f'''CREATE CUSTOM INDEX snippets_snippet_idx ON {keyspace}.snippets (snippet) USING 'StorageAttachedIndex'
  WITH OPTIONS = {{
    'index_analyzer': '{{
      "tokenizer": {{
        "name": "standard"
      }},
      "filters": [
        {{
          "name": "lowercase"
        }},
        {{
          "name": "porterstem"
        }}
      ]
    }}',
    'query_analyzer': 'keyword'
  }};'''

session.execute(CREATE_S_IDX)

#### A test query to see what this index finds

In [16]:
# we're not bothering with preparing statements here (variable shape, left as an exercise)
# Hence, note the '%s' in place of the '?'.

KEYWORD_QUERY_TEMPLATE = f"SELECT snippet FROM {keyspace}.snippets{{where_clause}} LIMIT %s ALLOW FILTERING"

def create_where_parts(keywords, placeholder='%s', logical_joiner='AND'):
    where_clause_pieces = [
        f"snippet : {placeholder}"
        for _ in sorted(set(keywords))
    ]
    where_clause_args = sorted(set(keywords))
    if where_clause_pieces:
        return (' WHERE ' + (f' {logical_joiner} ').join(where_clause_pieces), where_clause_args)
    else:
        return ('', list())

def find_by_keywords(keywords, n=3):
    wc, wc_vals = create_where_parts(keywords)
    #
    keyword_query = KEYWORD_QUERY_TEMPLATE.format(where_clause=wc)
    vals = tuple(wc_vals + [n])
    return [
        # let's pass a number to keep the output shape
        (row.snippet, 1.0)
        for row in session.execute(keyword_query, vals)
    ]

Note that "speak" matches "speaking" and "Speaking".

But also remember that this index is (purposefully) configured not to process the query, so you should not expect results when passing keywords such as `"Speak", "speaking", "having trouble"`.

In [17]:
KW = ['speak']
print(f"KEYWORDS: \'{', '.join(KW)}\'")
show(find_by_keywords(KW, 10))

KEYWORDS: 'speak'
    [1] 1.00000 "I cannot speak with the support operator!"
    [2] 1.00000 "Speaking to a technicial is impossible, WTF?"


In [18]:
show(find_by_keywords(['Speak'], 10))
show(find_by_keywords(['speaking'], 10))
show(find_by_keywords(['having trouble'], 10))

## Hybrid proper

Assume that "stemming + untouched query" is the right analyzers solution to go with vectors. Then it's time to combine the two for real.

There are several things to consider in approaching "hybrid search". The first choice can be stated as:
- vector condition and term condition are in AND or OR? In other words, do we run a single query or two queries and merge the results "code-side"?

Another point will be,
- explore re-ranking methods and form a "code-side" logic that combines the similarity coming from ANN with a computed keyword-matching-score. Possibly with user-supplied relative weights.
- This likely calls for a higher prefetch value and some client-side processing.

But what is our "keyword query"?
- Do we require _all_ keywords to be featured, or even a single one is ok (the more, the higher the 'score')?
- Possibly this could venture in the territory of arbitrary nesting of AND and OR with term predicates (**Not addressed yet**)

Last, important point. We will start with an interface where terms are supplied explicitly, but the final goal is a "magic box" where only the `query` is passed as input, the keyword being extracted behind the scenes from the query. Who does that? How is that done?
- We will address this point once the previous ones are better understood.

### Single-query approach (i.e. vector AND keywords)

We start with a "manual" approach, a function that accepts an explicit list of terms. And now we require the keyword predicate right in the vector search:

In [19]:
# Again, we don't bother with preparing statements in this demo - shapeshifter CQL statement.
HYBRID_ANN_QUERY_TEMPLATE = """
SELECT snippet, similarity_cosine(embedding, %s) as similarity
FROM {keyspace}.snippets
  {{where_clause}}
  ORDER BY embedding ANN OF %s
  LIMIT %s ;
""".format(keyspace=keyspace)

def hybrid_ann(query, keywords=[], top_k=3):
    q_vector = get_embeddings([query])[0]
    #
    wc, wc_vals = create_where_parts(keywords)
    #
    hybrid_query = HYBRID_ANN_QUERY_TEMPLATE.format(where_clause=wc)
    hq_values = tuple([q_vector] + wc_vals + [q_vector, top_k])
    return [
        (row.snippet, row.similarity)
        for row in session.execute(hybrid_query, hq_values)
    ]

In [20]:
KW3 = ['support']
QUERY3 = "How come I cannot chat?"
print(f"QUERY: '{QUERY3}', KEYWORDS: \'{', '.join(KW3)}\'")
show(hybrid_ann(QUERY3, KW3, 4))

QUERY: 'How come I cannot chat?', KEYWORDS: 'support'
    [1] 0.93516 "I cannot open the support chat."
    [2] 0.92320 "I see no messages in the support chat."
    [3] 0.91677 "I cannot speak with the support operator!"
    [4] 0.90999 "The support chat on the website is lagging."


In [21]:
KW4 = ['support', 'chat']
QUERY4 = "How come I cannot chat?"
print(f"QUERY: '{QUERY4}', KEYWORDS: \'{', '.join(KW4)}\'")
show(hybrid_ann(QUERY4, KW4, 8))

QUERY: 'How come I cannot chat?', KEYWORDS: 'support, chat'
    [1] 0.93515 "I cannot open the support chat."
    [2] 0.92317 "I see no messages in the support chat."
    [3] 0.90996 "The support chat on the website is lagging."


### Single-query, but with keywords in OR

This is interesting. We want to count as matches rows which have _at least one_ of the supplied keywords. (this is why we equipped the where-creation utility with a parameter for the logical joiner, defaulting to "and"...)

In [22]:
def hybrid_ann_anykeyword(query, keywords=[], top_k=3):
    q_vector = get_embeddings([query])[0]
    #
    wc, wc_vals = create_where_parts(keywords, logical_joiner='OR')
    #
    hybrid_query = HYBRID_ANN_QUERY_TEMPLATE.format(where_clause=wc)
    hq_values = tuple([q_vector] + wc_vals + [q_vector, top_k])
    return [
        (row.snippet, row.similarity)
        for row in session.execute(hybrid_query, hq_values)
    ]

You see differences from the AND case when having more than one keyword (but notice the similarity is vector-only):

In [23]:
KW5 = ['support', 'chat']
QUERY5 = "How come I cannot chat?"
print(f"QUERY: '{QUERY5}', KEYWORDS: \'{', '.join(KW5)}\'")
show(hybrid_ann_anykeyword(QUERY5, KW5, 8))

QUERY: 'How come I cannot chat?', KEYWORDS: 'support, chat'
    [1] 0.93515 "I cannot open the support chat."
    [2] 0.93222 "A message disappeared from the chat?"
    [3] 0.92317 "I see no messages in the support chat."
    [4] 0.91675 "I cannot speak with the support operator!"
    [5] 0.91518 "An operator chats with several people at the same time?"
    [6] 0.90996 "The support chat on the website is lagging."


### The quest for an overall "hybrid score"

We want a similarity that combines the keyword requirements as well. Something simple like

```
score = (1-rho) * vector_similarity + rho * keyword_similarity
```

or

```
score = vector_similarity^(1-rho) + keyword_similarity^rho
```

where the two similarities are thought to be between zero and one (or narrower in practice). Also the "keyword importance" `rho` parameter ideally is user-supplied during a phase of "tuning to the problem domain" (and is assumed to lie in `(0:1)`).

Let's come up with a generic postprocessor that we want to apply to any specific search technique:

In [25]:
# Draft implementations, don't take these too seriously for real texts

PUNKT = set('!,.?;\'"-+=/[]{}()\n')

def keyword_similarity(snippet, keywords=[], min=0.0, max=1.0):
    # if no keywords, we always return the max - to avoid breaking simpler flows
    if not keywords:
        return max
    else:
        _snp = ''.join([c for c in snippet if c not in PUNKT]).lower()
        toks = {tk for tk in _snp.split(' ') if tk}
        kw_set = set(keywords)
        num_kw = len(kw_set)
        num_hits = len(toks & kw_set)
        return min + (max - min) * num_hits / num_kw

In [29]:
# A little test
print(keyword_similarity("The quick brown fox ... wait, that's too long!", ["fox", "deer", "long"]))

print(keyword_similarity("The quick brown fox ... wait, that's too long!", ["fox", "deer", "long"], min=0.8))

0.6666666666666666
0.9333333333333333


In [27]:
def sum_score_merger(ann_sim, kw_sim, rho=0.5):
    return (1-rho)*ann_sim + rho*kw_sim


def combine_ann_with_kw_similarity(ann_results, keywords, kw_similarity_function=keyword_similarity, score_merger_function=sum_score_merger):
    # kw_similarity_function must accept a (text, kw_list) signature - possibly through partialing
    return sorted(
        (
            (
                snp,
                (score_merger_function(ann_sim, kw_similarity_function(snp, keywords))),
            )
            for snp, ann_sim in ann_results
        ),
        key=itemgetter(1),
        reverse=True,
    )

Let's retry the last hybrid query, aiming at a more balanced way to pick the "top" results:

In [30]:
# utility function
def hybrid_ann_anykw_fullscore(query, keywords=[], top_k=3):
    return combine_ann_with_kw_similarity(
        hybrid_ann_anykeyword(query, keywords, top_k),
        keywords,
    )

In [31]:
KW5 = ['support', 'chat']
QUERY5 = "How come I cannot chat?"
print(f"QUERY: '{QUERY5}', KEYWORDS: \'{', '.join(KW5)}\'")
show(hybrid_ann_anykw_fullscore(QUERY5, KW5, 8))

QUERY: 'How come I cannot chat?', KEYWORDS: 'support, chat'
    [1] 0.96757 "I cannot open the support chat."
    [2] 0.96157 "I see no messages in the support chat."
    [3] 0.95497 "The support chat on the website is lagging."
    [4] 0.71610 "A message disappeared from the chat?"
    [5] 0.70836 "I cannot speak with the support operator!"
    [6] 0.45759 "An operator chats with several people at the same time?"


Now the results with _both_ keywords are at the top. But perhaps the gap is too high?

Knowledge of the problem domain, and the typical matches, might lead to tuning the involved parameters for optimal results, e.g.:

In [32]:
my_keyword_similarity = partial(keyword_similarity, min=0.95)
my_sum_score_merger = partial(sum_score_merger, rho=0.2)


def my_hybrid_ann_anykw_fullscore(query, keywords=[], top_k=3):
    return combine_ann_with_kw_similarity(
        hybrid_ann_anykeyword(query, keywords, top_k),
        keywords,
        kw_similarity_function=my_keyword_similarity,
        score_merger_function=my_sum_score_merger,
    )

In [33]:
KW6 = ['support', 'chat']
QUERY6 = "How come I cannot chat?"
print(f"QUERY: '{QUERY6}', KEYWORDS: \'{', '.join(KW6)}\'")
show(my_hybrid_ann_anykw_fullscore(QUERY6, KW6, 8))

QUERY: 'How come I cannot chat?', KEYWORDS: 'support, chat'
    [1] 0.94810 "I cannot open the support chat."
    [2] 0.94073 "A message disappeared from the chat?"
    [3] 0.93850 "I see no messages in the support chat."
    [4] 0.92837 "I cannot speak with the support operator!"
    [5] 0.92794 "The support chat on the website is lagging."
    [6] 0.92212 "An operator chats with several people at the same time?"


Note the different order, coming from a new definition of the "full similarity".

But then, with the re-ranking occurring on "code-side", it is entirely possible that results that would rank higher are left out of the ANN query in the first place! A better definition would entail some sort of "safe-enough" higher prefetch `k`, and a later in-code cut to the required `top_k` (with the introduction of a "prefetch factor" constant to be kept high enough):

In [34]:
# one function to rule them all...
def hybrid_ann_anykw_fullscore_prefetch(query, keywords=[], top_k=3, kw_similarity_function=keyword_similarity,
                                        score_merger_function=sum_score_merger, prefetch_factor=5):
    prefetch_k = prefetch_factor * top_k if keywords else top_k
    # Warning: wasteful implementation, that's not the point :)
    return combine_ann_with_kw_similarity(
        hybrid_ann_anykeyword(query, keywords, prefetch_k),
        keywords,
        kw_similarity_function=kw_similarity_function,
        score_merger_function=score_merger_function,
    )[:top_k]

As predicted, not doing the prefetch (demonstrated in the second query below) favors results that would rightfully be surpassed by more deserving hits:

In [35]:
KW7 = ['support', 'chat']
QUERY7 = "How come I cannot chat?"
print(f"[with safe prefetch] QUERY: '{QUERY7}', KEYWORDS: \'{', '.join(KW7)}\'")
show(hybrid_ann_anykw_fullscore_prefetch(QUERY7, KW7, 3))

print(f"\n[without safe prefetch] QUERY: '{QUERY7}', KEYWORDS: \'{', '.join(KW7)}\'")
show(hybrid_ann_anykw_fullscore_prefetch(QUERY7, KW7, 3, prefetch_factor=1))

[with safe prefetch] QUERY: 'How come I cannot chat?', KEYWORDS: 'support, chat'
    [1] 0.96756 "I cannot open the support chat."
    [2] 0.96156 "I see no messages in the support chat."
    [3] 0.95496 "The support chat on the website is lagging."

[without safe prefetch] QUERY: 'How come I cannot chat?', KEYWORDS: 'support, chat'
    [1] 0.96758 "I cannot open the support chat."
    [2] 0.96158 "I see no messages in the support chat."
    [3] 0.71610 "A message disappeared from the chat?"


### Summary so far

We have a "hybrid search" tool that:
- takes semantic _and_ keywords into account in searching
- combines the score into an overall "similarity" assessment
- thanks to prefetching, mostly ensures the right hits are found
- is easily configurable as for the parameters involved. These largely depend on the problem domain and the dataset (and are: `rho`, max/min for the keyword score, and the prefetch factor).
- Also the _functional form_ for computing the keyword score and mixing it to the semantic similarity can be customized with reasonable effort.

Remember, if you want to require _all_ keywords to be found, most of this machinery is moot and you can go back to using the plain `hybrid_ann`.

_(with little additional coding effort, you can handle less obvious requirements as well. If, for example, you need at least _two_ keywords for a match, you can customize the similarity and the score merger functions to yield a zero if only one is encountered, ...)_

Note that the above `hybrid_ann_anykw_fullscore_prefetch` also works as a "simple ANN" tool (there's also a little optimization that avoids the prefetch if no keywords are supplied):

The only caveat is that you'd get different _numbers_ out-of-the-box (but: same results in the same order).

You would need to adjust the parameters to reconstruct the very same numbers.

In [36]:
QUERY8 = "I am looking for cheap stuff"
print(f"[hybrid_ann] QUERY: '{QUERY8}'")
show(hybrid_ann(QUERY8))

print(f"\n[hybrid_ann_anykw_fullscore_prefetch] QUERY: '{QUERY8}'")
show(hybrid_ann_anykw_fullscore_prefetch(QUERY8))

print(f"\n[hybrid_ann_anykw_fullscore_prefetch, scale adjusted] QUERY: '{QUERY8}'")
show(hybrid_ann_anykw_fullscore_prefetch(
    query=QUERY8,
    score_merger_function=partial(sum_score_merger, rho=0.0)
))

[hybrid_ann] QUERY: 'I am looking for cheap stuff'
    [1] 0.91006 "I would like to buy gift cards. Where can I get discounts?"
    [2] 0.89789 "I want to inquire about a specific product line."
    [3] 0.89473 "Are special offers available?"

[hybrid_ann_anykw_fullscore_prefetch] QUERY: 'I am looking for cheap stuff'
    [1] 0.95502 "I would like to buy gift cards. Where can I get discounts?"
    [2] 0.94893 "I want to inquire about a specific product line."
    [3] 0.94736 "Are special offers available?"

[hybrid_ann_anykw_fullscore_prefetch, scale adjusted] QUERY: 'I am looking for cheap stuff'
    [1] 0.91006 "I would like to buy gift cards. Where can I get discounts?"
    [2] 0.89789 "I want to inquire about a specific product line."
    [3] 0.89473 "Are special offers available?"


### This is it for the time being.

In the next notebook, we'll start from the `hybrid_ann_anykw_fullscore_prefetch` and worry about the upper layer
in the stack, namely _how to automate keyword extraction from the supplied query_.

## Note: multiple-query approach

Running and merging two independent queries (keyword-only and vector-only) is **postponed for the time being**.

Notes to keep in mind in that case:
- we want to avoid duplicates when merging
- we need to be careful in (1) keeping the signature compatible with the above manpulations/postprocessing, when applicable, (2) figuring out if, and how, there is a "similarity" to compare the two types of match.